# Experimenting to fix cracks & folds in basil  
Using the Optimizer to fix cracks & folds in basil

In [1]:
!mkdir -p ~/.cloudvolume/secrets
!cp /notebooks/*.json ~/.cloudvolume/secrets/
!pip install cloud-volume
!pip install scikit-image

You are using pip version 9.0.3, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 9.0.3, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
from cloudvolume import CloudVolume
from cloudvolume.lib import Bbox, Vec
import numpy as np
import torch
from torch.autograd import Variable
from optimize import Optimizer
from copy import deepcopy
from skimage.transform import resize
from scipy.ndimage.morphology import binary_erosion
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
def use_src_mask_and_one_dst():
    img_mip = 2
    mask_mip = 5
    src = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/unmasked", mip=img_mip, 
                                                                    parallel=2, fill_missing=True)
    cracks = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/mask/crack_detector_v3", mip=mask_mip,
                                                                     parallel=2, fill_missing=True)
    folds = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/mask/fold_detector_v1", mip=mask_mip,
                                                                     parallel=2, fill_missing=True)
    flow = Optimizer(ndownsamples=5, currn=5, avgn=20, lambda1=0.5, lr=0.1, eps=0.001, min_iter=100, max_iter=5000)
    full_offset = np.array([102716, 107077, 526])
    full_size = np.array([2048, 2048, 1])
    full_bbox = Bbox(full_offset, full_offset+full_size)
    src_bbox = Bbox.from_slices(src.slices_from_global_coords(full_bbox))
    dst_bbox = src_bbox + [0,0,1]
    mask_bbox = Bbox.from_slices(folds.slices_from_global_coords(full_bbox))
    print(src_bbox)
    print(dst_bbox)
    print(mask_bbox)
    dst = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/optimizer_tests/v01_with_mask", mip=img_mip, 
                        info=src.info, parallel=False, fill_missing=True, non_aligned_writes=True, cdn_cache=False)
    dst.commit_info();
    dst[dst_bbox.to_slices()] = src[dst_bbox.to_slices()]
    src_img = src[src_bbox.to_slices()][:,:,0,0] / 255.0
    dst_img = src[dst_bbox.to_slices()][:,:,0,0] / 255.0

    fold = folds[mask_bbox.to_slices()][:,:,0,0] / 255.0
    crack = cracks[mask_bbox.to_slices()][:,:,0,0] / 255.0
    mask = resize(np.minimum(fold, crack), src_bbox.size3()[:2])
    field = flow.process(src_img, [dst_img], mask, [np.ones_like(dst_img)])
    pred = flow.render(src_img, field)
    dst[src_bbox.to_slices()] = (pred[0,0,:,:,np.newaxis, np.newaxis]*255).astype(np.uint8)
    

In [4]:
def use_masks_and_two_dsts():
    img_mip = 2
    mask_mip = 5
    src = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/unmasked", mip=img_mip, 
                                                                    parallel=2, fill_missing=True)
    cracks = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/mask/crack_detector_v3", mip=mask_mip,
                                                                     parallel=2, fill_missing=True)
    folds = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/mask/fold_detector_v1", mip=mask_mip,
                                                                     parallel=2, fill_missing=True)    
    flow = Optimizer(ndownsamples=5, currn=5, avgn=20, lambda1=0.5, lr=0.1, eps=0.001, min_iter=100, max_iter=5000)
    full_src_offset = np.array([102716, 107077, 526])
    full_src_size = np.array([2048, 2048, 1])
    full_src_bbox = Bbox(full_src_offset, full_src_offset+full_src_size)
    src_bbox = Bbox.from_slices(src.slices_from_global_coords(full_src_bbox))
    src_mask_bbox = Bbox.from_slices(folds.slices_from_global_coords(full_src_bbox))
    dst_bboxes = []
    dst_mask_bboxes = []
    for z in [-1,1]:
        full_dst_bbox = full_bbox + [0,0,z]
        dst_bbox = Bbox.from_slices(src.slices_from_global_coords(full_dst_bbox))
        dst_mask_bbox = Bbox.from_slices(folds.slices_from_global_coords(full_dst_bbox))
        dst_bboxes.append(dst_bbox)
        dst_mask_bboxes.append(dst_mask_bbox)
        
    dst = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/optimizer_tests/v01_mask_two_dsts", 
          mip=img_mip, info=src.info, parallel=False, fill_missing=True, non_aligned_writes=True, cdn_cache=False)
    dst.commit_info();
    
    dst_images = []
    dst_masks = []
    for dst_bbox, dst_mask_bbox in zip(dst_bboxes, dst_mask_bboxes):
        dst_slices = dst_bbox.to_slices()
        mask_slices = dst_mask_bbox.to_slices()
        dst[dst_slices] = src[dst_slices]
        
        dst_images.append(src[dst_slices][:,:,0,0] / 255.0)
        dst_fold = folds[mask_slices][:,:,0,0] / 255.0
        dst_crack = cracks[mask_slices][:,:,0,0] / 255.0
        dst_mask = resize(np.minimum(dst_fold, dst_crack), dst_bbox.size3()[:2])
        dst_masks.append(dst_mask)
        
        
    src_image = src[src_bbox.to_slices()][:,:,0,0] / 255.0
    src_fold = folds[src_mask_bbox.to_slices()][:,:,0,0] / 255.0
    src_crack = cracks[src_mask_bbox.to_slices()][:,:,0,0] / 255.0
    src_mask = resize(np.minimum(src_fold, src_crack), src_bbox.size3()[:2])
    
    field = flow.process(src_image, dst_images, src_mask, dst_masks)
    pred = flow.render(src_image, field)
    dst[src_bbox.to_slices()] = (pred[0,0,:,:,np.newaxis, np.newaxis]*255).astype(np.uint8)
    

In [5]:
def use_masks_and_four_dsts():
    img_mip = 2
    mask_mip = 5
    src = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/unmasked", mip=img_mip, 
                                                                    parallel=2, fill_missing=True)
    cracks = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/mask/crack_detector_v3", mip=mask_mip,
                                                                     parallel=2, fill_missing=True)
    folds = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/mask/fold_detector_v1", mip=mask_mip,
                                                                     parallel=2, fill_missing=True)    
    flow = Optimizer(ndownsamples=5, currn=5, avgn=20, lambda1=0.5, lr=0.1, eps=0.001, min_iter=100, max_iter=5000)
    full_src_offset = np.array([102716, 107077, 526])
    full_src_size = np.array([2048, 2048, 1])
    full_src_bbox = Bbox(full_src_offset, full_src_offset+full_src_size)
    src_bbox = Bbox.from_slices(src.slices_from_global_coords(full_src_bbox))
    src_mask_bbox = Bbox.from_slices(folds.slices_from_global_coords(full_src_bbox))
    dst_bboxes = []
    dst_mask_bboxes = []
    for z in [-2,-1,1,2]:
        full_dst_bbox = full_bbox + [0,0,z]
        dst_bbox = Bbox.from_slices(src.slices_from_global_coords(full_dst_bbox))
        dst_mask_bbox = Bbox.from_slices(folds.slices_from_global_coords(full_dst_bbox))
        dst_bboxes.append(dst_bbox)
        dst_mask_bboxes.append(dst_mask_bbox)
        
    dst = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/optimizer_tests/v01_mask_four_dsts", 
          mip=img_mip, info=src.info, parallel=False, fill_missing=True, non_aligned_writes=True, cdn_cache=False)
    dst.commit_info();
    
    dst_images = []
    dst_masks = []
    for dst_bbox, dst_mask_bbox in zip(dst_bboxes, dst_mask_bboxes):
        dst_slices = dst_bbox.to_slices()
        mask_slices = dst_mask_bbox.to_slices()
        dst[dst_slices] = src[dst_slices]
        
        dst_images.append(src[dst_slices][:,:,0,0] / 255.0)
        dst_fold = folds[mask_slices][:,:,0,0] / 255.0
        dst_crack = cracks[mask_slices][:,:,0,0] / 255.0
        dst_mask = resize(np.minimum(dst_fold, dst_crack), dst_bbox.size3()[:2])
        dst_masks.append(dst_mask)
        
        
    src_image = src[src_bbox.to_slices()][:,:,0,0] / 255.0
    src_fold = folds[src_mask_bbox.to_slices()][:,:,0,0] / 255.0
    src_crack = cracks[src_mask_bbox.to_slices()][:,:,0,0] / 255.0
    src_mask = resize(np.minimum(src_fold, src_crack), src_bbox.size3()[:2])
    
    field = flow.process(src_image, dst_images, src_mask, dst_masks)
    pred = flow.render(src_image, field)
    dst[src_bbox.to_slices()] = (pred[0,0,:,:,np.newaxis, np.newaxis]*255).astype(np.uint8)
    

In [6]:
def use_filtered_mask_and_two_dsts():
    img_mip = 2
    mask_mip = 5
    src = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/unmasked", mip=img_mip, 
                                                                    parallel=2, fill_missing=True)
    cracks = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/mask/crack_detector_v3", mip=mask_mip,
                                                                     parallel=2, fill_missing=True)
    folds = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/mask/fold_detector_v1", mip=mask_mip,
                                                                     parallel=2, fill_missing=True)    
    flow = Optimizer(ndownsamples=5, currn=5, avgn=20, lambda1=0.5, lr=0.1, eps=0.001, min_iter=100, max_iter=5000)
    full_src_offset = np.array([102716, 107077, 526])
    full_src_size = np.array([2048, 2048, 1])
    full_src_bbox = Bbox(full_src_offset, full_src_offset+full_src_size)
    src_bbox = Bbox.from_slices(src.slices_from_global_coords(full_src_bbox))
    src_mask_bbox = Bbox.from_slices(folds.slices_from_global_coords(full_src_bbox))
    dst_bboxes = []
    dst_mask_bboxes = []
    for z in [-1,1]:
        full_dst_bbox = full_bbox + [0,0,z]
        dst_bbox = Bbox.from_slices(src.slices_from_global_coords(full_dst_bbox))
        dst_mask_bbox = Bbox.from_slices(folds.slices_from_global_coords(full_dst_bbox))
        dst_bboxes.append(dst_bbox)
        dst_mask_bboxes.append(dst_mask_bbox)
        
    dst = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/optimizer_tests/v01_filtered_mask_two_dsts", 
          mip=img_mip, info=src.info, parallel=False, fill_missing=True, non_aligned_writes=True, cdn_cache=False)
    dst.commit_info();
    
    dst_images = []
    dst_masks = []
    for dst_bbox, dst_mask_bbox in zip(dst_bboxes, dst_mask_bboxes):
        dst_slices = dst_bbox.to_slices()
        mask_slices = dst_mask_bbox.to_slices()
        dst[dst_slices] = src[dst_slices]
        
        dst_images.append(src[dst_slices][:,:,0,0] / 255.0)
        dst_fold = folds[mask_slices][:,:,0,0] / 255.0
        dst_crack = cracks[mask_slices][:,:,0,0] / 255.0
        dst_mask = resize(np.minimum(dst_fold, dst_crack), dst_bbox.size3()[:2])
        dst_masks.append(dst_mask)
        
        
    src_image = src[src_bbox.to_slices()][:,:,0,0] / 255.0
    src_fold = folds[src_mask_bbox.to_slices()][:,:,0,0] / 255.0
    src_crack = cracks[src_mask_bbox.to_slices()][:,:,0,0] / 255.0
    src_mask = resize(np.minimum(src_fold, src_crack), src_bbox.size3()[:2])
    
    src_mask *= (src_mask < 1/255.0) | (src_mask > 10/255.0)
    
    field = flow.process(src_image, dst_images, src_mask, dst_masks)
    pred = flow.render(src_image, field)
    dst[src_bbox.to_slices()] = (pred[0,0,:,:,np.newaxis, np.newaxis]*255).astype(np.uint8)
    

In [7]:
def use_filtered_mask_and_two_dsts2():
    img_mip = 2
    mask_mip = 5
    src = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/unmasked", mip=img_mip, 
                                                                    parallel=2, fill_missing=True)
    cracks = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/mask/crack_detector_v3", mip=mask_mip,
                                                                     parallel=2, fill_missing=True)
    folds = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/mask/fold_detector_v1", mip=mask_mip,
                                                                     parallel=2, fill_missing=True)    
    flow = Optimizer(ndownsamples=5, currn=5, avgn=20, lambda1=0.5, lr=0.1, eps=0.001, min_iter=100, max_iter=5000)
    full_src_offset = np.array([102716, 107077, 526])
    full_src_size = np.array([2048, 2048, 1])
    full_src_bbox = Bbox(full_src_offset, full_src_offset+full_src_size)
    src_bbox = Bbox.from_slices(src.slices_from_global_coords(full_src_bbox))
    src_mask_bbox = Bbox.from_slices(folds.slices_from_global_coords(full_src_bbox))
    dst_bboxes = []
    dst_mask_bboxes = []
    for z in [-1,1]:
        full_dst_bbox = full_bbox + [0,0,z]
        dst_bbox = Bbox.from_slices(src.slices_from_global_coords(full_dst_bbox))
        dst_mask_bbox = Bbox.from_slices(folds.slices_from_global_coords(full_dst_bbox))
        dst_bboxes.append(dst_bbox)
        dst_mask_bboxes.append(dst_mask_bbox)
        
    dst = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/optimizer_tests/v01_filtered_mask_two_dsts2", 
          mip=img_mip, info=src.info, parallel=False, fill_missing=True, non_aligned_writes=True, cdn_cache=False)
    dst.commit_info();
    
    dst_images = []
    dst_masks = []
    for dst_bbox, dst_mask_bbox in zip(dst_bboxes, dst_mask_bboxes):
        dst_slices = dst_bbox.to_slices()
        mask_slices = dst_mask_bbox.to_slices()
        dst[dst_slices] = src[dst_slices]
        
        dst_images.append(src[dst_slices][:,:,0,0] / 255.0)
        dst_fold = folds[mask_slices][:,:,0,0] / 255.0
        dst_crack = cracks[mask_slices][:,:,0,0] / 255.0
        dst_mask = resize(np.minimum(dst_fold, dst_crack), dst_bbox.size3()[:2])
        dst_masks.append(dst_mask)
        
        
    src_image = src[src_bbox.to_slices()][:,:,0,0] / 255.0
    src_fold = folds[src_mask_bbox.to_slices()][:,:,0,0] / 255.0
    src_crack = cracks[src_mask_bbox.to_slices()][:,:,0,0] / 255.0
    src_mask = resize(np.minimum(src_fold, src_crack), src_bbox.size3()[:2])
    
    src_mask *= (src_mask < 4/255.0) | (src_mask > 10/255.0)
    
    field = flow.process(src_image, dst_images, src_mask, dst_masks)
    pred = flow.render(src_image, field)
    dst[src_bbox.to_slices()] = (pred[0,0,:,:,np.newaxis, np.newaxis]*255).astype(np.uint8)
    

In [8]:
def use_masks_and_two_dsts_mip1():
    img_mip = 1
    mask_mip = 5
    src = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/unmasked", mip=img_mip, 
                                                                    parallel=2, fill_missing=True)
    cracks = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/mask/crack_detector_v3", mip=mask_mip,
                                                                     parallel=2, fill_missing=True)
    folds = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/mask/fold_detector_v1", mip=mask_mip,
                                                                     parallel=2, fill_missing=True)
    flow = Optimizer(ndownsamples=5, currn=5, avgn=20, lambda1=0.5, lr=0.1, eps=0.001, min_iter=100, max_iter=5000)
    full_src_offset = np.array([102716, 107077, 526])
    full_src_size = np.array([2048, 2048, 1])
    full_src_bbox = Bbox(full_src_offset, full_src_offset+full_src_size)
    src_bbox = Bbox.from_slices(src.slices_from_global_coords(full_src_bbox))
    src_mask_bbox = Bbox.from_slices(folds.slices_from_global_coords(full_src_bbox))
    dst_bboxes = []
    dst_mask_bboxes = []
    for z in [-1,1]:
        full_dst_bbox = full_src_bbox + [0,0,z]
        dst_bbox = Bbox.from_slices(src.slices_from_global_coords(full_dst_bbox))
        dst_mask_bbox = Bbox.from_slices(folds.slices_from_global_coords(full_dst_bbox))
        dst_bboxes.append(dst_bbox)
        dst_mask_bboxes.append(dst_mask_bbox)
        
    dst = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/optimizer_tests/v01_mask_two_dsts_mip1", 
          mip=img_mip, info=src.info, parallel=False, fill_missing=True, non_aligned_writes=True, cdn_cache=False)
    dst.commit_info();
    
    dst_images = []
    dst_masks = []
    for dst_bbox, dst_mask_bbox in zip(dst_bboxes, dst_mask_bboxes):
        dst_slices = dst_bbox.to_slices()
        mask_slices = dst_mask_bbox.to_slices()
        dst[dst_slices] = src[dst_slices]
        
        dst_images.append(src[dst_slices][:,:,0,0] / 255.0)
        dst_fold = folds[mask_slices][:,:,0,0] / 255.0
        dst_crack = cracks[mask_slices][:,:,0,0] / 255.0
        dst_mask = resize(np.minimum(dst_fold, dst_crack), dst_bbox.size3()[:2])
        dst_masks.append(dst_mask)
        
        
    src_image = src[src_bbox.to_slices()][:,:,0,0] / 255.0
    src_fold = folds[src_mask_bbox.to_slices()][:,:,0,0] / 255.0
    src_crack = cracks[src_mask_bbox.to_slices()][:,:,0,0] / 255.0
    src_mask = resize(np.minimum(src_fold, src_crack), src_bbox.size3()[:2])
    
    field = flow.process(src_image, dst_images, src_mask, dst_masks)
    pred = flow.render(src_image, field)
    dst[src_bbox.to_slices()] = (pred[0,0,:,:,np.newaxis, np.newaxis]*255).astype(np.uint8)
    return field
    

In [9]:
def use_masks_and_two_dsts_mip0():
    img_mip = 0
    mask_mip = 5
    src = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/unmasked", mip=img_mip, 
                                                                    parallel=2, fill_missing=True)
    cracks = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/mask/crack_detector_v3", mip=mask_mip,
                                                                     parallel=2, fill_missing=True)
    folds = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/mask/fold_detector_v1", mip=mask_mip,
                                                                     parallel=2, fill_missing=True)
    flow = Optimizer(ndownsamples=5, currn=5, avgn=20, lambda1=0.5, lr=0.1, eps=0.001, min_iter=100, max_iter=5000)
    full_src_offset = np.array([102716, 107077, 526])
    full_src_size = np.array([2048, 2048, 1])
    full_src_bbox = Bbox(full_src_offset, full_src_offset+full_src_size)
    src_bbox = Bbox.from_slices(src.slices_from_global_coords(full_src_bbox))
    src_mask_bbox = Bbox.from_slices(folds.slices_from_global_coords(full_src_bbox))
    dst_bboxes = []
    dst_mask_bboxes = []
    for z in [-1,1]:
        full_dst_bbox = full_src_bbox + [0,0,z]
        dst_bbox = Bbox.from_slices(src.slices_from_global_coords(full_dst_bbox))
        dst_mask_bbox = Bbox.from_slices(folds.slices_from_global_coords(full_dst_bbox))
        dst_bboxes.append(dst_bbox)
        dst_mask_bboxes.append(dst_mask_bbox)
        
    dst = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/optimizer_tests/v01_mask_two_dsts_mip0", 
          mip=img_mip, info=src.info, parallel=False, fill_missing=True, non_aligned_writes=True, cdn_cache=False)
    dst.commit_info();
    
    dst_images = []
    dst_masks = []
    for dst_bbox, dst_mask_bbox in zip(dst_bboxes, dst_mask_bboxes):
        dst_slices = dst_bbox.to_slices()
        mask_slices = dst_mask_bbox.to_slices()
        dst[dst_slices] = src[dst_slices]
        
        dst_images.append(src[dst_slices][:,:,0,0] / 255.0)
        dst_fold = folds[mask_slices][:,:,0,0] / 255.0
        dst_crack = cracks[mask_slices][:,:,0,0] / 255.0
        dst_mask = resize(np.minimum(dst_fold, dst_crack), dst_bbox.size3()[:2])
        dst_masks.append(dst_mask)
        
        
    src_image = src[src_bbox.to_slices()][:,:,0,0] / 255.0
    src_fold = folds[src_mask_bbox.to_slices()][:,:,0,0] / 255.0
    src_crack = cracks[src_mask_bbox.to_slices()][:,:,0,0] / 255.0
    src_mask = resize(np.minimum(src_fold, src_crack), src_bbox.size3()[:2])
    
    field = flow.process(src_image, dst_images, src_mask, dst_masks)
    pred = flow.render(src_image, field)
    dst[src_bbox.to_slices()] = (pred[0,0,:,:,np.newaxis, np.newaxis]*255).astype(np.uint8)
    return field
    

In [10]:
def use_masks_and_two_dsts_crack():
    img_mip = 2
    mask_mip = 5
    src = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/unmasked", mip=img_mip, 
                                                                    parallel=2, fill_missing=True)
    cracks = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/mask/crack_detector_v3", mip=mask_mip,
                                                                     parallel=2, fill_missing=True)
    folds = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/mask/fold_detector_v1", mip=mask_mip,
                                                                     parallel=2, fill_missing=True)    
    flow = Optimizer(ndownsamples=5, currn=5, avgn=20, lambda1=0.5, lr=0.1, eps=0.001, min_iter=100, max_iter=5000)
    full_src_offset = np.array([102320, 111106, 526])
    full_src_size = np.array([2048, 2048, 1])
    full_src_bbox = Bbox(full_src_offset, full_src_offset+full_src_size)
    src_bbox = Bbox.from_slices(src.slices_from_global_coords(full_src_bbox))
    src_mask_bbox = Bbox.from_slices(folds.slices_from_global_coords(full_src_bbox))
    dst_bboxes = []
    dst_mask_bboxes = []
    for z in [-1,1]:
        full_dst_bbox = full_src_bbox + [0,0,z]
        dst_bbox = Bbox.from_slices(src.slices_from_global_coords(full_dst_bbox))
        dst_mask_bbox = Bbox.from_slices(folds.slices_from_global_coords(full_dst_bbox))
        dst_bboxes.append(dst_bbox)
        dst_mask_bboxes.append(dst_mask_bbox)
        
    dst = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/optimizer_tests/v01_mask_two_dsts", 
          mip=img_mip, info=src.info, parallel=False, fill_missing=True, non_aligned_writes=True, cdn_cache=False)
    dst.commit_info();
    
    dst_images = []
    dst_masks = []
    for dst_bbox, dst_mask_bbox in zip(dst_bboxes, dst_mask_bboxes):
        dst_slices = dst_bbox.to_slices()
        mask_slices = dst_mask_bbox.to_slices()
        dst[dst_slices] = src[dst_slices]
        
        dst_images.append(src[dst_slices][:,:,0,0] / 255.0)
        dst_fold = folds[mask_slices][:,:,0,0] / 255.0
        dst_crack = cracks[mask_slices][:,:,0,0] / 255.0
        dst_mask = resize(np.minimum(dst_fold, dst_crack), dst_bbox.size3()[:2])
        dst_masks.append(dst_mask)
        
        
    src_image = src[src_bbox.to_slices()][:,:,0,0] / 255.0
    src_fold = folds[src_mask_bbox.to_slices()][:,:,0,0] / 255.0
    src_crack = cracks[src_mask_bbox.to_slices()][:,:,0,0] / 255.0
    src_mask = resize(np.minimum(src_fold, src_crack), src_bbox.size3()[:2])
    
    src_mask = np.ones_like(src_mask) * ((src_mask < 6/255.0) | (src_mask > 20/255.0))
    
    field = flow.process(src_image, dst_images, src_mask, dst_masks)
    pred = flow.render(src_image, field)
    dst[src_bbox.to_slices()] = (pred[0,0,:,:,np.newaxis, np.newaxis]*255).astype(np.uint8)
    return field

In [11]:
def use_gt_masks_and_two_dsts():
    mip = 2
    src = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/unmasked", mip=mip, 
                                                                    parallel=2, fill_missing=True)
    mask = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/mask/ground_truth", mip=mip,
                                                                     parallel=2, fill_missing=True)
    dst = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/optimizer_tests/v02_ground_truth_masks_two_dsts", 
          mip=mip, info=src.info, parallel=False, fill_missing=True, non_aligned_writes=True, cdn_cache=False)
    dst.commit_info();
    flow = Optimizer(ndownsamples=5, currn=5, avgn=20, lambda1=0.5, lr=0.1, eps=0.001, min_iter=100, max_iter=5000)
    offsets = [Vec(102320, 111106, 526), (102716, 107077, 526)]
    full_src_size = np.array([2048, 2048, 1])
    for full_src_offset in offsets:
        full_src_bbox = Bbox(full_src_offset, full_src_offset+full_src_size)
        src_bbox = Bbox.from_slices(src.slices_from_global_coords(full_src_bbox))
        dst_bboxes = []
        for z in [-1,1]:
            full_dst_bbox = full_src_bbox + [0,0,z]
            dst_bbox = Bbox.from_slices(src.slices_from_global_coords(full_dst_bbox))
            dst_bboxes.append(dst_bbox)

        dst_images = []
        dst_masks = []
        for dst_bbox in dst_bboxes:
            dst_slices = dst_bbox.to_slices()
            dst[dst_slices] = src[dst_slices]

            dst_images.append(src[dst_slices][:,:,0,0] / 255.0)
            dst_mask = (mask[dst_slices][:,:,0,0] == 0) / 1.0
            dst_masks.append(dst_mask)


        src_image = src[src_bbox.to_slices()][:,:,0,0] / 255.0
        src_mask = (mask[src_bbox.to_slices()][:,:,0,0] == 0) / 1.0

        field = flow.process(src_image, dst_images, src_mask, dst_masks)
        pred = flow.render(src_image, field)
        dst[src_bbox.to_slices()] = (pred[0,0,:,:,np.newaxis, np.newaxis]*255).astype(np.uint8)

In [12]:
def without_masks_and_two_dsts():
    mip = 2
    src = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/unmasked", mip=mip, 
                                                                    parallel=2, fill_missing=True)
    mask = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/mask/ground_truth", mip=mip,
                                                                     parallel=2, fill_missing=True)
    dst = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/optimizer_tests/v02_without_masks_two_dsts", 
          mip=mip, info=src.info, parallel=False, fill_missing=True, non_aligned_writes=True, cdn_cache=False)
    dst.commit_info();
    flow = Optimizer(ndownsamples=5, currn=5, avgn=20, lambda1=0.5, lr=0.1, eps=0.001, min_iter=100, max_iter=5000)
    offsets = [Vec(102320, 111106, 526), (102716, 107077, 526)]
    full_src_size = np.array([2048, 2048, 1])
    for full_src_offset in offsets:
        full_src_bbox = Bbox(full_src_offset, full_src_offset+full_src_size)
        src_bbox = Bbox.from_slices(src.slices_from_global_coords(full_src_bbox))
        dst_bboxes = []
        for z in [-1,1]:
            full_dst_bbox = full_src_bbox + [0,0,z]
            dst_bbox = Bbox.from_slices(src.slices_from_global_coords(full_dst_bbox))
            dst_bboxes.append(dst_bbox)

        dst_images = []
        dst_masks = []
        for dst_bbox in dst_bboxes:
            dst_slices = dst_bbox.to_slices()
            dst[dst_slices] = src[dst_slices]

            dst_image = src[dst_slices][:,:,0,0] / 255.0
            dst_images.append(dst_image)
            # dst_mask = (mask[dst_slices][:,:,0,0] == 0) / 1.0
            dst_mask = np.ones_like(dst_image)
            dst_masks.append(dst_mask)


        src_image = src[src_bbox.to_slices()][:,:,0,0] / 255.0
        # src_mask = (mask[src_bbox.to_slices()][:,:,0,0] == 0) / 1.0
        src_mask = np.ones_like(src_image)

        field = flow.process(src_image, dst_images, src_mask, dst_masks)
        pred = flow.render(src_image, field)
        dst[src_bbox.to_slices()] = (pred[0,0,:,:,np.newaxis, np.newaxis]*255).astype(np.uint8)

In [13]:
def use_masks_and_two_dsts_more_folds():
    img_mip = 2
    mask_mip = 5
    src = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/unmasked", mip=img_mip, 
                                                                    parallel=2, fill_missing=True)
    cracks = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/mask/crack_detector_v3", mip=mask_mip,
                                                                     parallel=2, fill_missing=True)
    folds = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/mask/fold_detector_v1", mip=mask_mip,
                                                                     parallel=2, fill_missing=True)    
    dst = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/optimizer_tests/v01_mask_two_dsts", 
          mip=img_mip, info=src.info, parallel=False, fill_missing=True, non_aligned_writes=True, cdn_cache=False)
    dst.commit_info();
    flow = Optimizer(ndownsamples=5, currn=5, avgn=20, lambda1=0.5, lr=0.1, eps=0.001, min_iter=100, max_iter=5000)
    offsets = [Vec(191721, 139068, 528),    # consecutive folds
               Vec(196097, 177069, 526),    # consecutive folds
               Vec(179301, 81627, 529),     # big crack
               Vec(167572, 73149, 522),     # track mark, no masks
               Vec(146350, 204341, 524),    # consecutive cracks
               Vec(130159, 162051, 525)     # consecutive cracks
              ]
    full_src_size = np.array([2048, 2048, 1])
    for full_src_offset in offsets:
        full_src_bbox = Bbox(full_src_offset, full_src_offset+full_src_size)
        src_bbox = Bbox.from_slices(src.slices_from_global_coords(full_src_bbox))
        src_mask_bbox = Bbox.from_slices(folds.slices_from_global_coords(full_src_bbox))
        dst_bboxes = []
        dst_mask_bboxes = []
        for z in [-1,1]:
            full_dst_bbox = full_src_bbox + [0,0,z]
            dst_bbox = Bbox.from_slices(src.slices_from_global_coords(full_dst_bbox))
            dst_mask_bbox = Bbox.from_slices(folds.slices_from_global_coords(full_dst_bbox))
            dst_bboxes.append(dst_bbox)
            dst_mask_bboxes.append(dst_mask_bbox)

        dst_images = []
        dst_masks = []
        for dst_bbox, dst_mask_bbox in zip(dst_bboxes, dst_mask_bboxes):
            dst_slices = dst_bbox.to_slices()
            mask_slices = dst_mask_bbox.to_slices()
            dst[dst_slices] = src[dst_slices]

            dst_images.append(src[dst_slices][:,:,0,0] / 255.0)
            dst_fold = folds[mask_slices][:,:,0,0] / 255.0
            dst_crack = cracks[mask_slices][:,:,0,0] / 255.0
            crack_max = np.max(dst_crack * (dst_crack < 20/255.0))
            crack_min = max(1, crack_max-3)
            dst_crack = np.ones_like(dst_crack) * ((dst_crack < crack_min) | (dst_crack > crack_max))
            dst_mask = resize(np.minimum(dst_fold, dst_crack), dst_bbox.size3()[:2])
            dst_masks.append(dst_mask)


        src_image = src[src_bbox.to_slices()][:,:,0,0] / 255.0
        src_fold = folds[src_mask_bbox.to_slices()][:,:,0,0] / 255.0
        src_crack = cracks[src_mask_bbox.to_slices()][:,:,0,0] / 255.0
        crack_max = np.max(src_crack * (src_crack < 20/255.0))
        crack_min = max(1, crack_max-3)
        src_crack = np.ones_like(src_crack) * ((src_crack < crack_min) | (src_crack > crack_max))
        src_mask = resize(np.minimum(src_fold, src_crack), src_bbox.size3()[:2])

        field = flow.process(src_image, dst_images, src_mask, dst_masks)
        pred = flow.render(src_image, field)
        dst[src_bbox.to_slices()] = (pred[0,0,:,:,np.newaxis, np.newaxis]*255).astype(np.uint8)
    

In [14]:
def erode_mask(mask, radius=300, th=50):
    """Generate bitmask by thresholding mask below th, then eroding by radius
    """
    bitmask = mask < th
    f = np.ones((radius, radius))
    bitmask = binary_erosion(bitmask, structure=f, border_value=1)
    return ~bitmask*1

In [15]:
def use_masks_and_two_dsts_more_cracks():
    img_mip = 2
    mask_mip = 5
    radius = 250
    src = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/unmasked", mip=img_mip, 
                                                                    parallel=2, fill_missing=True)
    cracks = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/mask/crack_detector_v3", mip=mask_mip,
                                                                     parallel=2, fill_missing=True)
    folds = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/mask/fold_detector_v1", mip=mask_mip,
                                                                     parallel=2, fill_missing=True)    
    dst = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/optimizer_tests/v01_mask_two_dsts", 
          mip=img_mip, info=src.info, parallel=False, fill_missing=True, non_aligned_writes=True, cdn_cache=False)
    dst.commit_info();
    flow = Optimizer(ndownsamples=5, currn=5, avgn=20, lambda1=0.5, lr=0.1, eps=0.001, min_iter=100, max_iter=5000)
    offsets = [Vec(179301, 81627, 529) - Vec(1024, 1024, 0),     # big crack
               Vec(146350, 204341, 525),                         # consecutive cracks
               Vec(130159, 162051, 525)                          # consecutive cracks
              ]
    sizes = [Vec(3072, 3072, 1),
             Vec(2048, 2048, 1),
             Vec(2048, 2048, 1)]
    for full_src_offset, full_src_size in zip(offsets, sizes):
        full_src_bbox = Bbox(full_src_offset, full_src_offset+full_src_size)
        src_bbox = Bbox.from_slices(src.slices_from_global_coords(full_src_bbox))
        src_mask_bbox = Bbox.from_slices(folds.slices_from_global_coords(full_src_bbox))
        dst_bboxes = []
        dst_mask_bboxes = []
        for z in [-1,1]:
            full_dst_bbox = full_src_bbox + [0,0,z]
            dst_bbox = Bbox.from_slices(src.slices_from_global_coords(full_dst_bbox))
            dst_mask_bbox = Bbox.from_slices(folds.slices_from_global_coords(full_dst_bbox))
            dst_bboxes.append(dst_bbox)
            dst_mask_bboxes.append(dst_mask_bbox)

        dst_images = []
        dst_masks = []
        for dst_bbox, dst_mask_bbox in zip(dst_bboxes, dst_mask_bboxes):
            dst_slices = dst_bbox.to_slices()
            mask_slices = dst_mask_bbox.to_slices()
            dst[dst_slices] = src[dst_slices]

            dst_images.append(src[dst_slices][:,:,0,0] / 255.0)
            dst_fold = folds[mask_slices][:,:,0,0] / 255.0
            dst_crack = cracks[mask_slices][:,:,0,0]
            dst_crack = erode_mask(dst_crack, radius=int(round(radius/2**mask_mip)))
            dst_mask = resize(np.minimum(dst_fold, dst_crack), dst_bbox.size3()[:2])
            dst_masks.append(dst_mask)


        src_image = src[src_bbox.to_slices()][:,:,0,0] / 255.0
        src_fold = folds[src_mask_bbox.to_slices()][:,:,0,0] / 255.0
        src_crack = cracks[src_mask_bbox.to_slices()][:,:,0,0]
        src_crack = erode_mask(src_crack, radius=int(round(radius/2**mask_mip)))
        src_mask = resize(np.minimum(src_fold, src_crack), src_bbox.size3()[:2])

        field = flow.process(src_image, dst_images, src_mask, dst_masks)
        pred = flow.render(src_image, field)
        dst[src_bbox.to_slices()] = (pred[0,0,:,:,np.newaxis, np.newaxis]*255).astype(np.uint8)
    

In [16]:
def use_masks_and_two_dsts_more_cracks_downsample6():
    img_mip = 2
    mask_mip = 5
    radius = 250
    src = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/unmasked", mip=img_mip, 
                                                                    parallel=2, fill_missing=True)
    cracks = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/mask/crack_detector_v3", mip=mask_mip,
                                                                     parallel=2, fill_missing=True)
    folds = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/mask/fold_detector_v1", mip=mask_mip,
                                                                     parallel=2, fill_missing=True)    
    dst = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/optimizer_tests/v01_mask_two_dsts_ndownsamples6", 
          mip=img_mip, info=src.info, parallel=False, fill_missing=True, non_aligned_writes=True, cdn_cache=False)
    dst.commit_info();
    flow = Optimizer(ndownsamples=6, currn=6, avgn=20, lambda1=0.5, lr=0.1, eps=0.001, min_iter=100, max_iter=5000)
    offsets = [Vec(179301, 81627, 529) - Vec(1024, 1024, 0),     # big crack
               Vec(146350, 204341, 525),                         # consecutive cracks
               Vec(130159, 162051, 525)                          # consecutive cracks
              ]
    sizes = [Vec(3072, 3072, 1),
             Vec(2048, 2048, 1),
             Vec(2048, 2048, 1)]
    for full_src_offset, full_src_size in zip(offsets, sizes):
        full_src_bbox = Bbox(full_src_offset, full_src_offset+full_src_size)
        src_bbox = Bbox.from_slices(src.slices_from_global_coords(full_src_bbox))
        src_mask_bbox = Bbox.from_slices(folds.slices_from_global_coords(full_src_bbox))
        dst_bboxes = []
        dst_mask_bboxes = []
        for z in [-1,1]:
            full_dst_bbox = full_src_bbox + [0,0,z]
            dst_bbox = Bbox.from_slices(src.slices_from_global_coords(full_dst_bbox))
            dst_mask_bbox = Bbox.from_slices(folds.slices_from_global_coords(full_dst_bbox))
            dst_bboxes.append(dst_bbox)
            dst_mask_bboxes.append(dst_mask_bbox)

        dst_images = []
        dst_masks = []
        for dst_bbox, dst_mask_bbox in zip(dst_bboxes, dst_mask_bboxes):
            dst_slices = dst_bbox.to_slices()
            mask_slices = dst_mask_bbox.to_slices()
            dst[dst_slices] = src[dst_slices]

            dst_images.append(src[dst_slices][:,:,0,0] / 255.0)
            dst_fold = folds[mask_slices][:,:,0,0] / 255.0
            dst_crack = cracks[mask_slices][:,:,0,0]
            dst_crack = erode_mask(dst_crack, radius=int(round(radius/2**mask_mip)))
            dst_mask = resize(np.minimum(dst_fold, dst_crack), dst_bbox.size3()[:2])
            dst_masks.append(dst_mask)


        src_image = src[src_bbox.to_slices()][:,:,0,0] / 255.0
        src_fold = folds[src_mask_bbox.to_slices()][:,:,0,0] / 255.0
        src_crack = cracks[src_mask_bbox.to_slices()][:,:,0,0]
        src_crack = erode_mask(src_crack, radius=int(round(radius/2**mask_mip)))
        src_mask = resize(np.minimum(src_fold, src_crack), src_bbox.size3()[:2])

        field = flow.process(src_image, dst_images, src_mask, dst_masks)
        pred = flow.render(src_image, field)
        dst[src_bbox.to_slices()] = (pred[0,0,:,:,np.newaxis, np.newaxis]*255).astype(np.uint8)
    

In [75]:
def make_field(dim, k=-0.005):
    field = torch.zeros((1,dim,dim,2))
    for i in range(dim):
        for j in range(dim):
            if i < dim - j:
                field[0,i,j,0] = 1*k
                field[0,i,j,1] = 1*k
            else:
                field[0,i,j,0] = -1*k
                field[0,i,j,1] = -1*k
    return field

In [103]:
def use_masks_and_two_dsts_big_crack_initialized_field():
    img_mip = 2
    mask_mip = 5
    radius = 250
    ndownsamples = 5
    src = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/unmasked", mip=img_mip, 
                                                                    parallel=2, fill_missing=True)
    cracks = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/mask/crack_detector_v3", mip=mask_mip,
                                                                     parallel=2, fill_missing=True)
    folds = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/mask/fold_detector_v1", mip=mask_mip,
                                                                     parallel=2, fill_missing=True)    
    dst = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/optimizer_tests/v01_mask_two_dsts_initialized_field", 
          mip=img_mip, info=src.info, parallel=False, fill_missing=True, non_aligned_writes=True, cdn_cache=False)
    dst.commit_info();
    flow = Optimizer(ndownsamples=ndownsamples, currn=ndownsamples, avgn=20, 
                             lambda1=0.9, lr=0.1, eps=0.001, min_iter=100, max_iter=5000)
    offsets = [Vec(179301, 81627, 529) - Vec(1024, 1024, 0)]
    sizes = [Vec(3072, 3072, 1)]
    field = 0
    for full_src_offset, full_src_size in zip(offsets, sizes):
        full_src_bbox = Bbox(full_src_offset, full_src_offset+full_src_size)
        src_bbox = Bbox.from_slices(src.slices_from_global_coords(full_src_bbox))
        src_mask_bbox = Bbox.from_slices(folds.slices_from_global_coords(full_src_bbox))
        dst_bboxes = []
        dst_mask_bboxes = []
        for z in [-1,1]:
            full_dst_bbox = full_src_bbox + [0,0,z]
            dst_bbox = Bbox.from_slices(src.slices_from_global_coords(full_dst_bbox))
            dst_mask_bbox = Bbox.from_slices(folds.slices_from_global_coords(full_dst_bbox))
            dst_bboxes.append(dst_bbox)
            dst_mask_bboxes.append(dst_mask_bbox)

        dst_images = []
        dst_masks = []
        for dst_bbox, dst_mask_bbox in zip(dst_bboxes, dst_mask_bboxes):
            dst_slices = dst_bbox.to_slices()
            mask_slices = dst_mask_bbox.to_slices()
            dst[dst_slices] = src[dst_slices]

            dst_images.append(src[dst_slices][:,:,0,0] / 255.0)
            dst_fold = folds[mask_slices][:,:,0,0] / 255.0
            dst_crack = cracks[mask_slices][:,:,0,0]
            dst_crack = erode_mask(dst_crack, radius=int(round(radius/2**mask_mip)))
            dst_mask = resize(np.minimum(dst_fold, dst_crack), dst_bbox.size3()[:2])
            dst_masks.append(dst_mask)


        src_image = src[src_bbox.to_slices()][:,:,0,0] / 255.0
        src_fold = folds[src_mask_bbox.to_slices()][:,:,0,0] / 255.0
        # src_crack = cracks[src_mask_bbox.to_slices()][:,:,0,0]
        # src_crack = erode_mask(src_crack, radius=int(round(radius/2**mask_mip)))
        # src_mask = resize(np.minimum(src_fold, src_crack), src_bbox.size3()[:2])
        src_crack = src_image < (210.0 / 255.0)
        src_mask = resize(src_crack, src_bbox.size3()[:2])

        dim = int(src_image.shape[-1] / (2**(ndownsamples - 1)))
        field = make_field(dim)
        # field = None
        field = flow.process(src_image, dst_images, src_mask, dst_masks, field=field)
        pred = flow.render(src_image, field)
        dst[src_bbox.to_slices()] = (pred[0,0,:,:,np.newaxis, np.newaxis]*255).astype(np.uint8)
    return field

In [101]:
def use_masks_and_two_dsts_big_crack_uninitialized_field():
    img_mip = 2
    mask_mip = 5
    radius = 250
    ndownsamples = 5
    src = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/unmasked", mip=img_mip, 
                                                                    parallel=2, fill_missing=True)
    cracks = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/mask/crack_detector_v3", mip=mask_mip,
                                                                     parallel=2, fill_missing=True)
    folds = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/mask/fold_detector_v1", mip=mask_mip,
                                                                     parallel=2, fill_missing=True)    
    dst = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/optimizer_tests/v01_mask_two_dsts_uninitialized_field", 
          mip=img_mip, info=src.info, parallel=False, fill_missing=True, non_aligned_writes=True, cdn_cache=False)
    dst.commit_info();
    flow = Optimizer(ndownsamples=ndownsamples, currn=ndownsamples, avgn=20, 
                             lambda1=0.9, lr=0.1, eps=0.001, min_iter=100, max_iter=5000)
    offsets = [Vec(179301, 81627, 529) - Vec(1024, 1024, 0)]
    sizes = [Vec(3072, 3072, 1)]
    field = 0
    for full_src_offset, full_src_size in zip(offsets, sizes):
        full_src_bbox = Bbox(full_src_offset, full_src_offset+full_src_size)
        src_bbox = Bbox.from_slices(src.slices_from_global_coords(full_src_bbox))
        src_mask_bbox = Bbox.from_slices(folds.slices_from_global_coords(full_src_bbox))
        dst_bboxes = []
        dst_mask_bboxes = []
        for z in [-1,1]:
            full_dst_bbox = full_src_bbox + [0,0,z]
            dst_bbox = Bbox.from_slices(src.slices_from_global_coords(full_dst_bbox))
            dst_mask_bbox = Bbox.from_slices(folds.slices_from_global_coords(full_dst_bbox))
            dst_bboxes.append(dst_bbox)
            dst_mask_bboxes.append(dst_mask_bbox)

        dst_images = []
        dst_masks = []
        for dst_bbox, dst_mask_bbox in zip(dst_bboxes, dst_mask_bboxes):
            dst_slices = dst_bbox.to_slices()
            mask_slices = dst_mask_bbox.to_slices()
            dst[dst_slices] = src[dst_slices]

            dst_images.append(src[dst_slices][:,:,0,0] / 255.0)
            dst_fold = folds[mask_slices][:,:,0,0] / 255.0
            dst_crack = cracks[mask_slices][:,:,0,0]
            dst_crack = erode_mask(dst_crack, radius=int(round(radius/2**mask_mip)))
            dst_mask = resize(np.minimum(dst_fold, dst_crack), dst_bbox.size3()[:2])
            dst_masks.append(dst_mask)


        src_image = src[src_bbox.to_slices()][:,:,0,0] / 255.0
        src_fold = folds[src_mask_bbox.to_slices()][:,:,0,0] / 255.0
        # src_crack = cracks[src_mask_bbox.to_slices()][:,:,0,0]
        # src_crack = erode_mask(src_crack, radius=int(round(radius/2**mask_mip)))
        # src_mask = resize(np.minimum(src_fold, src_crack), src_bbox.size3()[:2])
        src_crack = src_image < (210.0 / 255.0)
        src_mask = resize(src_crack, src_bbox.size3()[:2])

        # dim = int(src_image.shape[-1] / (2**(ndownsamples - 1)))
        # field = make_field(dim)
        field = None
        field = flow.process(src_image, dst_images, src_mask, dst_masks, field=field)
        pred = flow.render(src_image, field)
        dst[src_bbox.to_slices()] = (pred[0,0,:,:,np.newaxis, np.newaxis]*255).astype(np.uint8)
    return field

In [104]:
use_masks_and_two_dsts_big_crack_initialized_field()

Downloading:   0%|          | 0/2 [00:00<?, ?it/s]

Rechunking image: 0it [00:00, ?it/s]
Uploading: 0it [00:00, ?it/s]
Rechunking image:   0%|          | 0/1 [00:00<?, ?it/s]

Rechunking image:   0%|          | 0/1 [00:00<?, ?it/s]


Rechunking image: 100%|██████████| 1/1 [00:00<00:00, 11.28it/s]


Rechunking image:   0%|          | 0/1 [00:00<?, ?it/s]

Rechunking image: 100%|██████████| 1/1 [00:00<00:00,  9.30it/s]


Rechunking image: 100%|██████████| 1/1 [00:00<00:00,  4.95it/s]

Rechunking image: 100%|██████████| 1/1 [00:00<00:00,  6.48it/s]

Downloading:   0%|          | 0/2 [00:00<?, ?it/s]

Downloading:   0%|          | 0/1 [00:00<?, ?it/s]
/usr/local/lib/python3.5/dist-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
Downloading:   0%|          | 0/2 [00:00<?, ?it/s]

Rechunking image: 0it [00:00, ?it/s]
Uploading: 0it [00:00, 

(768, 768) 2 (768, 768) 2
Downsample factor 16x
[0.34358537]
[0.3346164]
[0.32766324]
[0.32193166]
[0.3170181]
[0.31263492]
[0.30874255]
[0.3053571]
[0.30235308]
[0.2997125]
[0.29736024]
[0.295239]
[0.2933209]
[0.29154095]
[0.28998363]
[0.28850657]
[0.28713748]
[0.2858995]
[0.28459457]
[0.2834314]
[0.28232402]
[0.28134507]
[0.28041154]
[0.27955356]
[0.27862284]
[0.27779448]
[0.27702868]
[0.2763089]
[0.2756658]
[0.2750085]
[0.27445126]
[0.27382708]
[0.27324712]
[0.27271158]
[0.27219546]
[0.27176556]
[0.27130145]
[0.27086717]
[0.27049136]
[0.27006933]
[0.26972955]
[0.26932672]
[0.26894313]
[0.2686527]
[0.26828945]
[0.26797283]
[0.26765895]
[0.26732066]
[0.26706612]
[0.266761]
[0.2664711]
[0.26623517]
[0.2659729]
[0.26575327]
[0.2654565]
[0.26523536]
[0.26504695]
[0.2647947]
[0.26459548]
[0.26435545]
[0.26417232]
[0.26399273]
[0.2637862]
[0.2635763]
[0.26339656]
[0.2632336]
[0.26303732]
[0.26288325]
[0.26267838]
[0.262526]
[0.26237518]
[0.26218754]
[0.26203898]
[0.26190192]
[0.26172197]
[

[0.24444306]
[0.24443331]
[0.24441385]
[0.24443498]
[0.24440761]
[0.24440467]
[0.24440455]
[0.24442281]
[0.24438891]
[0.24437568]
[0.24437338]
[0.24438824]
[0.24439539]
[0.24435677]
[0.24435867]
[0.24438354]
[0.24434716]
[0.2443592]
[0.24432695]
[0.24435884]
[0.244321]
[0.2443443]
[0.2442997]
[0.24434143]
[0.24428985]
[0.24433447]
[0.24431351]
[0.2443058]
[0.24429728]
[0.24429558]
[0.24429986]
[0.2442565]
[0.24430126]
[0.244259]
[0.24427053]
[0.24425709]
[0.24425703]
[0.24425897]
[0.244241]
[0.24423629]
[0.24423894]
[0.24424726]
[0.24422318]
[0.24425147]
[0.24418074]
[0.24424934]
[0.24418426]
[0.24421673]
[0.24418247]
[0.24419124]
[0.24420102]
[0.24420303]
[0.24418359]
[0.24416363]
[0.24419776]
[0.24415801]
[0.2441844]
[0.24414445]
[0.24414884]
[0.24416845]
[0.24415836]
[0.24412653]
[0.24413314]
[0.24414563]
[0.24413276]
[0.24415499]
[0.24409166]
[0.24413586]
[0.24412563]
[0.24411403]
[0.24410665]
[0.24409948]
[0.24408557]
[0.24409725]
[0.24409273]
[0.24408448]
[0.24410297]
[0.24408142

[0.37236887]
[0.3723522]
[0.3723171]
[0.37228608]
[0.37229124]
[0.3722514]
[0.37223923]
[0.37221748]
[0.37222958]
[0.3721717]
[0.3721905]
[0.37215877]
[0.372122]
[0.37210858]
[0.3721248]
[0.3720951]
[0.3720771]
[0.37203652]
[0.37205455]
[0.37202844]
[0.37202173]
[0.37200594]
[0.37197533]
[0.3719639]
[0.37195402]
[0.37195522]
[0.371898]
[0.3719175]
[0.3718831]
[0.3718661]
[0.37188706]
[0.3718732]
[0.37183243]
[0.3718282]
[0.37182695]
[0.37180695]
[0.37177137]
[0.37177378]
[0.37178054]
[0.37175137]
[0.37174636]
[0.37174165]
[0.37170118]
[0.37171426]
[0.37167835]
[0.37166083]
[0.37166226]
[0.37165636]
[0.37161154]
[0.37162596]
[0.37162495]
[0.37158364]
[0.3715879]
[0.37157094]
[0.37155885]
[0.37153578]
[0.37153974]
[0.37152436]
[0.37152755]
[0.37150025]
[0.37149155]
[0.37147945]
[0.37146273]
[0.37147206]
[0.37146026]
[0.3714298]
[0.37141615]
[0.37141517]
[0.37141088]
[0.37138045]
[0.37141228]
[0.37136766]
[0.37135905]
[0.37136427]
[0.37133813]
[0.37135303]
[0.37130383]
[0.3713218]
[0.3713

[0.83772266]
[0.8376881]
[0.8376652]
Downsample factor 1x
[1.6944891]
[1.2806373]
[1.2165916]
[1.1913841]
[1.1789923]
[1.171141]
[1.1649423]
[1.1603613]
[1.15563]
[1.1524508]
[1.1506217]
[1.1471374]
[1.1450598]
[1.1428429]
[1.1409876]
[1.1391912]
[1.1384509]
[1.1366423]
[1.1357069]
[1.1340795]
[1.1330912]
[1.1319048]
[1.1324126]
[1.1309769]
[1.130682]
[1.1297703]
[1.1293054]
[1.1287875]
[1.1282473]
[1.1271403]
[1.1273155]
[1.1267102]
[1.1267935]
[1.1262391]
[1.1263609]
[1.1248934]
[1.1254973]
[1.1246612]
[1.124995]
[1.1231744]
[1.1236973]
[1.1231651]
[1.1238325]
[1.1226667]
[1.1225319]
[1.1221015]
[1.1228223]
[1.1217701]
[1.1220404]
[1.1218445]
[1.1220236]
[1.121057]
[1.1214274]
[1.1208228]
[1.1208974]
[1.120588]
[1.120076]
[1.1200826]
[1.1205609]
[1.1200724]
[1.119868]
[1.1193255]
[1.11988]
[1.1199908]
[1.1195226]
[1.1188419]
[1.1191791]
[1.1183727]
[1.1184821]
[1.1180344]
[1.1182669]
[1.117733]
[1.1177089]
[1.1172857]
[1.1175256]
[1.1171701]
[1.1173475]
[1.1171482]
[1.1172745]
[1.116

Rechunking image: 0it [00:00, ?it/s]
Uploading: 0it [00:00, ?it/s]
Rechunking image:   0%|          | 0/1 [00:00<?, ?it/s]

Rechunking image:   0%|          | 0/1 [00:00<?, ?it/s]


Rechunking image:   0%|          | 0/1 [00:00<?, ?it/s]



Rechunking image:   0%|          | 0/1 [00:00<?, ?it/s]

[1.1156082]
done: 2132
torch.Size([1, 768, 768, 2])




Rechunking image: 100%|██████████| 1/1 [00:00<00:00,  8.23it/s]

Rechunking image: 100%|██████████| 1/1 [00:00<00:00,  6.95it/s]

Rechunking image: 100%|██████████| 1/1 [00:00<00:00,  6.73it/s]



array([[[-0.005     , -0.005     ],
        [-0.005     , -0.005     ],
        [-0.005     , -0.005     ],
        ...,
        [-0.005     , -0.005     ],
        [-0.005     , -0.005     ],
        [-0.005     , -0.005     ]],

       [[-0.005     , -0.005     ],
        [-0.005     , -0.005     ],
        [-0.005     , -0.005     ],
        ...,
        [-0.00446232, -0.00446232],
        [-0.00442477, -0.00442477],
        [-0.00438722, -0.00438722]],

       [[-0.005     , -0.005     ],
        [-0.005     , -0.005     ],
        [-0.005     , -0.005     ],
        ...,
        [-0.00392465, -0.00392465],
        [-0.00384955, -0.00384955],
        [-0.00377445, -0.00377445]],

       ...,

       [[-0.005     , -0.005     ],
        [-0.00446232, -0.00446232],
        [-0.00392465, -0.00392465],
        ...,
        [ 0.005     ,  0.005     ],
        [ 0.005     ,  0.005     ],
        [ 0.005     ,  0.005     ]],

       [[-0.005     , -0.005     ],
        [-0.00442477, -0.00